# Introduction


**Goal** : To classify whether a location is rice land or non rice land

**Why Rice** : Rice is the primary crop and food staple of more than half the world’s population. Asia is the world’s largest rice-producing and rice-consuming region.

<img src="Images/Rice Image.jpeg" alt="alt text" width="400" height="300" class="blog-image">


Researchers and government organizations routinely use satellite data to identify the location of agriculture crops and forecast yield. Unfortunately, there is no ideal solution that yields perfect results.With increasing publicly available sattelite data there is a steady progress happening in this field.

## Data Collection

Here we obtain data from the following 2 sattelites :

- The **Sentinel-1** mission(ESA) uses C-band **radar** at **10-meter** resolution with a single mission revisit every **12 days**.

- The **Sentinel 2** is an Earth observation mission from the Copernicus Programme(ESA) that systematically acquires **optical imagery** at high spatial resolution over land and coastal waters. There are 2 sattelites which provides revisit every **5 days** at a particular location.

<img src="Images/Sentinel-1_radar_vision_pillars.jpeg" alt="alt text" width="400" height="300" class="blog-image">


## Analysis Region 

This project will use data from the **An Giang province** in the Mekong Delta in **Vietnam**. In particular, rice crop yield data was collected for the period of **late-2021 to mid-2022** over the **Chau Phu, Chau Thanh and Thoai Son districts**. This is a dense rice crop region with a mixture of double and triple cropping cycles. For this project we will assume triple cropping (3 cycles per year) with a focus on the **November-21 to April-22 and the April to August-22**.This data is spread evenly across these districts with nearly full coverage of each district. 

<img src="Images/Vietnam overview.jpeg" alt="alt text" width="500" height="300" class="blog-image">  

## About the Data : 

The data was obtained from Microsoft Planetary hub through API. It was populated in the form of the data cube i.e. ndarray.

<img src="Images/Data Cube.jpeg" alt="alt text" width="500" height="400" class="blog-image">  



Training data : Latitude and Longitude( Data of approx 600 locations)

With the help of Stac(microsoft planetary hub) API , we obtain Optical data (e.g., Landsat or Sentinel-2).It contains many spectral bands **(e.g., Red, Green, Blue, Red-Edge, NIR, SWIR, etc.)** that can be related to the **presence or growth of rice crops**.Researchers often use **statistical combinations of these bands, called indices** to build models. One of the more common indices for agriculture is the **Normalized Difference Vegetation Index (NDVI)**, but there are many others including **Enhanced Vegetation Index (EVI)** and **Soil Adjusted Vegetation Index (SAVI)**. **Optical data is used to measure the "greenness" of vegetation during the growth cycle. Differences in band or index values can be used to distinguish differences in crop types.**

Training data : Latitude and Longitude( Data of approx 600 locations)

With the help of Stac(microsoft planetary hub) API , we obtain Optical data (e.g., Landsat or Sentinel-2).It contains many spectral bands **(e.g., Red, Green, Blue, Red-Edge, NIR, SWIR, etc.)** that can be related to the **presence or growth of rice crops**.Researchers often use **statistical combinations of these bands, called indices** to build models. One of the more common indices for agriculture is the **Normalized Difference Vegetation Index (NDVI)**, but there are many others including **Enhanced Vegetation Index (EVI)** and **Soil Adjusted Vegetation Index (SAVI)**. **Optical data is used to measure the "greenness" of vegetation during the growth cycle. Differences in band or index values can be used to distinguish differences in crop types.**

**NDVI (Normalized Difference Vegetation Index) = (NIR-Red) / (NIR+Red)**

## Rice Phenology

This figure gives us an overview of the rice growth.

<img src="Images/Rice stages.jpeg" alt="alt text" width="500" height="300" class="blog-image">

NVDI varies for every different surface rice crop,forest,river,orchad.

<img src="Images/NVDI real.jpeg" alt="alt text" width="600" height ="400" class ="blog-image">
<img src="Images/NVDI word.jpeg" alt="alt text" width="500" height="300" class="blog-image">

# Our Approach
The typical approach considers data from setinent 1 satellite (radar) which involves vv and vh bands. However, in our approach we wanted to experiment with sentient 2 satellite (optical) data and try to draw out accurate findings.  
Studying the rice phenology and dataset reveals that rice follows a **double cropping cycle in a year**. Each crop cycle has 3 main growing stages-   
- Vegetative  
- Reproductive  
- Ripening   
As shown above, the indices vary greatly during this time period. As our indices dataset is a time series data, we have divided dataset into 6 different parts i.e. **3 for each cycle**. This is our basic approach, we have further took 2 different approaches for feature selection.

## Approach 1:
We have considered multiple vegatative indices across all 6 time frames and calculated mean of each index for each time frame respective 

## Approach 2: 
We have considered a few most commonly used vegatative indices across all 6 time frames  then calculated mean as well as variance of each index for each time frame respective

# Data Processing

### Importing all the necessary Libraries important for processing and fetching the data

In [10]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import pandas as pd
import xarray as xr # library for working with labeled multidimensional arrays.

# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
import odc
from odc.stac import stac_load
from odc.algo import to_rgba
from shapely.geometry import box
import concurrent.futures


##############################################################################

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
import dask
from dask.distributed import Client as daskClient

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.impute import KNNImputer

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('df2f4d7a411942f8a5b81d60cb9b95ca')

# Others
import requests
import rich.table
import math
import random
from itertools import cycle
from tqdm import tqdm
from time import sleep
tqdm.pandas()

In [12]:
#Read the provided CSV file

get = pd.read_csv("Crop_Location_Data_20221201 (1).csv")
get.head(5)

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


In [ ]:
# Spererating Longitude and Laditude and its type to float

get[['Latitude','Longitude']] = get['Latitude and Longitude'].str.split(',',expand=True)
get['Latitude'] = get['Latitude'].str.replace(r'\(|\)', '', regex=True)
get['Longitude'] = get['Longitude'].str.replace(r'\(|\)', '', regex=True)

get['Latitude'] = get['Latitude'].astype(float)
get['Longitude'] = get['Longitude'].astype(float)
get


In [ ]:
#reference for what time slice to put

vegetative = "2021-11-01/2021-12-31"
reproductive = "2022-01-01/2022-02-28"
ripening = "2022-03-01/2022-04-30"

vegetative_2 = "2022-04-01/2022-05-31"
reproductive_2 = "2022-05-01/2022-06-30"
ripening_2 = "2022-07-01/2022-08-31"

### Feature Engineering

In [ ]:
# 

def process_lat_lon(lat, lon):
    box_size_deg = 0.0004 #5*5
    time_window = "2022-03-01/2022-04-30"
    resolution = 20
    scale = resolution / 111320.0

    # Calculate bounding box(min,max)
    bbox = box(lon - box_size_deg / 2, lat - box_size_deg / 2, lon + box_size_deg / 2, lat + box_size_deg / 2)
    
    # Fetching the data through Sentinel-2
    stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = stac.search(collections=["sentinel-2-l2a"], bbox=bbox.bounds, datetime=time_window)
    items = list(search.get_all_items())
    
    # Fetching the bands/variables in an xarray
    xx = stac_load(
        items,
        bands=["red", "green", "blue", "nir", "SCL", "B11", "B12"],
        crs="EPSG:4326",
        resolution=scale,
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=pc.sign,
        bbox=bbox.bounds
    )
    
    # Create a mask for no data, saturated data, clouds, cloud shadows, and water
    cloud_mask = \
        (xx.SCL != 0) & \
        (xx.SCL != 1) & \
        (xx.SCL != 3) & \
        (xx.SCL != 6) & \
        (xx.SCL != 8) & \
        (xx.SCL != 9) & \
        (xx.SCL != 10)
     
    # Apply cloud mask
    cleaned_data = xx.where(cloud_mask).astype("uint16")
    
    resample_period = '1w'
    window = 5
    
    
    # Calculate the mean of the data across the sample region and then NDVI
    mean_clean = cleaned_data.mean(dim=['longitude','latitude']).compute()
    ndvi_mean_clean = (mean_clean.nir - mean_clean.red) / (mean_clean.nir + mean_clean.red)
    interpolate_data = ndvi_mean_clean.to_dataframe(name='NDVI')
    # Apply the rolling window operation
    NDVI_interpolated = interpolate_data['NDVI'].resample(resample_period).median().rolling(window=window, min_periods=1).mean()

    # Calculate the mean of the data across the sample region and then EVI
    evi_mean_clean = 2.5 * ((mean_clean.nir - mean_clean.red) / (mean_clean.nir + (6 * mean_clean.red) + (7.5 * mean_clean.blue) + 1))
    interpolate_data = evi_mean_clean.to_dataframe(name='EVI')
    # Apply the rolling window operation
    EVI_interpolated = interpolate_data['EVI'].resample(resample_period).median().rolling(window=window, min_periods=1).mean()
                      
    # Calculate the mean of the data across the sample region and then RGVI
    rgvi_mean_clean = 1 - ((mean_clean.blue - mean_clean.red) / (mean_clean.nir + mean_clean.B11 + mean_clean.B12))
    interpolate_data = rgvi_mean_clean.to_dataframe(name='RGVI')
    # Apply the rolling window operation
    RGVI_interpolated = interpolate_data['RGVI'].resample(resample_period).median().rolling(window=window, min_periods=1).mean()
    
    # Calculate the mean of the data across the sample region and then NDBI
    ndbi_mean_clean = (mean_clean.B11 - mean_clean.nir) / (mean_clean.B11 + mean_clean.nir)
    interpolate_data = ndbi_mean_clean.to_dataframe(name='NDBI')
    # Apply the rolling window operation
    NDBI_interpolated = interpolate_data['NDBI'].resample(resample_period).median().rolling(window=window, min_periods=1).mean()
    
    # Calculate the mean of the data across the sample region and then DVI
    dvi_mean_clean = mean_clean.nir - mean_clean.red
    interpolate_data = dvi_mean_clean.to_dataframe(name='DVI')
    # Apply the rolling window operation
    DVI_interpolated = interpolate_data['DVI'].resample(resample_period).median().rolling(window=window, min_periods=1).mean()
    
    # Calculate the mean of the data across the sample region and then IPVI
    ipvi_mean_clean = (mean_clean.nir) / (mean_clean.nir + mean_clean.red)
    interpolate_data = ipvi_mean_clean.to_dataframe(name='IPVI')
    # Apply the rolling window operation
    IPVI_interpolated = interpolate_data['IPVI'].resample(resample_period).median().rolling(window=window, min_periods=1).mean()
    
    # Calculate the mean of the data across the sample region and then SAVI
    #SAVI is modified NVDI which is used to correct the brightness from soil
    L=0.5 #most commanly prefered value of correction factor
    savi_mean_clean = (1+L)*(mean_clean.nir-mean_clean.red) / (mean_clean.nir + mean_clean.red+L)
    interpolate_data = savi_mean_clean.to_dataframe(name='SAVI')
    # Apply the rolling window operation
    SAVI_interpolated = interpolate_data['SAVI'].resample(resample_period).median().rolling(window=window, min_periods=1).mean()
    
    # Calculate the mean of the data across the sample region and then MSAVI
    #MSAVI is modified MSAVI which is used to variable value of L depending on vegetation cover
    msavi_mean_clean = (2*mean_clean.nir+1-np.sqrt((2*mean_clean.nir)**2-8*(mean_clean.nir-mean_clean.red)))/2
    interpolate_data = msavi_mean_clean.to_dataframe(name='MSAVI')
    # Apply the rolling window operation
    MSAVI_interpolated = interpolate_data['MSAVI'].resample(resample_period).median().rolling(window=window, min_periods=1).mean()
    
    # Calculate the mean of the data across the sample region and then NDWI
    # Ususal for mapping water content on land; 0.5+ for water bodies
    ndwi_mean_clean = (mean_clean.green-mean_clean.nir)/(mean_clean.green+mean_clean.nir)
    interpolate_data = msavi_mean_clean.to_dataframe(name='NDWI')
    # Apply the rolling window operation
    NDWI_interpolated = interpolate_data['NDWI'].resample(resample_period).median().rolling(window=window, min_periods=1).mean()
    
    # Calculate the mean of the data across the sample region and then LSWI
    # Land surface water index
    lswi_mean_clean = (mean_clean.nir-mean_clean.B11)/(mean_clean.nir-mean_clean.B11)
    interpolate_data = lswi_mean_clean.to_dataframe(name='LSWI')
    # Apply the rolling window operation
    LSWI_interpolated = interpolate_data['LSWI'].resample(resample_period).median().rolling(window=window, min_periods=1).mean()
    
                   
    return {
        #mean 
        'NDVI': float(NDVI_interpolated.mean()),
        'EVI': float(EVI_interpolated.mean()),
        'RGVI': float(RGVI_interpolated.mean()),
        'NDBI': float(ndbi_mean_clean.mean()),
        'DVI': float(dvi_mean_clean.mean()),
        'IPVI': float(ipvi_mean_clean.mean()),
        'SAVI': float(SAVI_interpolated.mean()),
        'MSAVI': float(MSAVI_interpolated.mean()),
        'NDWI': float(NDWI_interpolated.mean()),
        'LSWI': float(LSWI_interpolated.mean()),
        #variances
        'NDVI_var': float(NDVI_interpolated.var()),
        'EVI_var': float(EVI_interpolated.var()),
        'RGVI_var': float(RGVI_interpolated.var()),
        'NDBI_var': float(ndbi_mean_clean.var()),
        'DVI_var': float(dvi_mean_clean.var()),
        'IPVI_var': float(ipvi_mean_clean.var()),
        'SAVI_var': float(SAVI_interpolated.var()),
        'MSAVI_var': float(MSAVI_interpolated.var()),
        'NDWI_var': float(NDWI_interpolated.var()),
        'LSWI_var': float(LSWI_interpolated.var())
    }


def process_lat_lon_batch(batch_df):
    results = []

    for index, row in batch_df.iterrows():
        lat = row['Latitude']
        lon = row['Longitude']

        results.append(process_lat_lon(lat, lon))

    return results

# Apply the processing function to latitude and longitude data in batches
batch_size = 2
lat_lon_batches = [get[['Latitude', 'Longitude']][i:i+batch_size] for i in range(0, len(get), batch_size)]

results = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    for result in executor.map(process_lat_lon_batch, lat_lon_batches):
        results.extend(result)
        
# Create a DataFrame from the results
result_df = pd.DataFrame(results)
result_df.to_csv('ripening.csv', index=False)
